In [30]:
%matplotlib qt
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [61]:
cam = cv.VideoCapture(0)
lower = np.array([100,180,150])
higher = np.array([150,255,255])
ret=True
prev = ''
while(ret):
    ret,frame = cam.read()
    #flip about y axis
    frame = cv.flip(frame,1)
    
    w = frame.shape[1]
    h = frame.shape[0]
    
    img_smooth = cv.GaussianBlur(frame,(7,7),0)
    
    #define ROI 
    #same shape as frame
    mask = np.zeros_like(frame)
    #create white region to extract that from frame
    mask[100:400,100:400] = (255,255,255)
    
    #creating rectange in the frame:
    cv.rectangle(frame,(100,100),(400,400),(0,0,255),3)
    cv.rectangle(frame,(200,100),(300,400),(0,0,255),2)
    cv.rectangle(frame,(100,200),(400,300),(0,0,255),2)
    
    img_roi = cv.bitwise_and(img_smooth, mask)

    img_hsv = cv.cvtColor(img_roi,cv.COLOR_BGR2HSV)
    #extracting pixels within specified range
    img_threshold = cv.inRange(img_hsv,lower,higher)
   
    contours,hierarchy = cv.findContours(img_threshold,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    #find contour with largest area (if contours present)
    if(len(contours)!=0):
        areas = [cv.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        #print(max_index)
        cnt = contours[max_index]
        
        #x_b,y_b,w_b,h_b = cv.boundingRect(cnt)
        #cv.rectangle(frame,(x_b,y_b),(x_b+w_b,y_b+h_b),(0,255,0),3)    
        #(100,100) (200,100) (300,100) (400,100)
        #(100,200)                     (400,200)
        #(100,300)                     (400,300)
        #(100,400) (200,400) (300,400) (400,400)
        M = cv.moments(cnt)
        if(M['m00']!=0):
            c_x = int(M['m10']/M['m00'])
            c_y = int(M['m01']/M['m00'])
            
            #finding location
            if(c_y<=200):
                #print('Upper ',end='')
                curr = 'Upper '
            elif(200<c_y<=300):
                #print('Middle ',end='')
                curr = 'Middle '
            elif(c_y>300):
                #print('Bottom ',end='')
                curr = 'Bottom '
                
            if(c_x<=200):
                #print('Left')
                curr = curr + 'Left'
            elif(200<c_x<=300):
                #print('Center')
                curr = curr + 'Center'
            elif(c_x>300):
                #print('Right')
                curr = curr + 'Right'
                 
            if(prev!=curr):
                print(curr)
                prev = curr
            center = (c_x,c_y)
            cv.circle(frame,center,4,(0,255,0),-1)
    else:
        if(len(prev)>0):
            print('Not in frame')
        prev = ''
    cv.imshow('Video',frame)
    if(cv.waitKey(10)==27):
        break
cam.release()
cv.destroyAllWindows()

Middle Right
Upper Right
Middle Right
Middle Center
Bottom Center
Bottom Left
Middle Left
Not in frame
Middle Left
Not in frame
Middle Left
Middle Center
Middle Right
Upper Right
Upper Center
Upper Left
Middle Left
Middle Center
Middle Right
Bottom Right
Bottom Center
Bottom Left
Middle Left
Middle Center
Not in frame
Upper Center
Upper Right
Middle Right
Bottom Right
Bottom Center
Bottom Left
Middle Center
Not in frame
Upper Center
Upper Right
Middle Right
Bottom Right
Not in frame
